## Custom image input to try out the ethnicity detector

In [2]:
import warnings
warnings.filterwarnings('ignore')
import cv2
import pandas as pd
import os
from os import listdir
from os.path import join, isfile, splitext, basename, isdir
import numpy as np
import dlib
import imutils
import matplotlib.pyplot as plt
import time
import math
from shutil import copyfile
import pickle
import gc

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import BatchNormalization, Input, Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Activation, AveragePooling2D, ZeroPadding2D, Concatenate, Reshape
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping #prevent losing model training since my computer shuts down alot when overheat
from tensorflow.keras.utils import to_categorical, Sequence

import matplotlib.pyplot as plt

from facial_landmark import FacialLandmark #this py file is created to help on this project
from facial_landmark_unopt import FacialLandmarkUnopt #unoptimized version
import models_collection

In [3]:
# For visualisation
def showImage(title, image):
    cv2.imshow(title, image)
    cv2.moveWindow(title,0,0)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def printImage(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.show()

In [4]:
class DataGenerator(Sequence):
    def __init__(self, list_IDs, labels, img_path, to_fit=True, batch_size=32, dim=(256,256), n_channels=3, n_classes=5, shuffle=True):
        self.list_IDs = list_IDs
        self.labels = labels
        self.img_path = img_path
        self.to_fit = to_fit
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        
        data = self._generate_data(list_IDs_temp)

        if self.to_fit:
            X, y = data
            return X, y
        else:
            return data
    
    def _generate_data(self, list_IDs):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        if self.to_fit:
            y = np.empty((self.batch_size), dtype=int)
#         print("loading", len(list_IDs), "images")
        for i, image_name in enumerate(list_IDs):
            pure_name = splitext(image_name)[0]
            split_pure_name = pure_name.split('_')
            if split_pure_name[-1] == "mirror":
                actual_image_name = "_".join(split_pure_name[:-1])+splitext(image_name)[1]
                image_path = join(self.img_path, actual_image_name)
                image_BGR_mir = cv2.imread(image_path)
                image_BGR = cv2.flip(image_BGR_mir, 1)
            else:
                image_path = join(self.img_path, image_name)
                image_BGR = cv2.imread(image_path)
            
            image_RGB = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2RGB)
            image_resized = cv2.resize(image_RGB, self.dim, interpolation = cv2.INTER_LINEAR)
            image_adjusted = image_resized / 255.0
            X[i,] = image_adjusted
            if self.to_fit:
                y[i,] = self.labels[image_name]
#         print("Done:", len(list_IDs),"Image loaded")
        if not self.to_fit:
            return X
        else:
            return (X,to_categorical(y, num_classes=self.n_classes))

In [5]:
model = load_model("./models_v2/AlexNet_optimized_v34/000_AlexNet_optimized_v34_best_loss_110.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [1]:
FILTERED_TEST_IMAGE_PATH = "./test_images"
image_name = "aj.jpg"
image = cv2.imread(image_name)
printImage(image)
y={}
y[image_name] = 2
facial_landmark = FacialLandmark(image)
face_only = facial_landmark.get_full_face_only()
save_path = join(FILTERED_TEST_IMAGE_PATH,image_name)
cv2.imwrite(save_path, face_only)
test_gen = DataGenerator([image_name],y, FILTERED_TEST_IMAGE_PATH, shuffle = False, batch_size=1)
label = ["white", "black", "asian", "indian", "others"]
pred = model.predict(test_gen, verbose=1)
print("prediction: ", label[np.argmax(pred[0])], pred[0] )